# Intro

the membrane-embedded nature of membrane proteins is not accounted for in traditional normal mode analysis.

As far as we can tell, two approaches have been implemented in [ProDy](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3309413/). The first one, [imANM](http://prody.csb.pitt.edu/tutorials/membrane_anm/imanm.html), separates the spring constant into 3 components and the component normal to the membrane is rescaled to dampen radial motions. The other one, [exANM](http://prody.csb.pitt.edu/tutorials/membrane_anm/exanm.html), models the whole slab of lipid membrane by space-filling it with beads, computes the Hessian of the whole system and reduces it yielding the effective Hessian of the protein in the membrane environment.

Although the approach in exANM is appealing, it is costly and requires to model the full membrane slab beforehand. We propose here a slightly different approach.

The idea would be to wrap the transmembrane region of the protein in a mesh of pseudo-particles harmonically constrained to stay in place and tightly coupled to the neighbouring protein atoms. Here also, the total Hessian is reduced to yield the effective Hessian of the protein, but this time at a much reduced cost.

# Methods

## Building the membrane skin

The molecular surface of the transmembrane part of the protein is found using a big probe radius, (e.g. see algorithm in  [epock](https://www.ncbi.nlm.nih.gov/pubmed/25505095) [code](https://bitbucket.org/epock/epock)) and the voxels identified are kept as anchor-particles.

## Building the Hessian

The total Hessian can be seen as the block matrix $H=\left[\begin{array}{c c} H_{PP} & H_{PM} \\ H_{MP} & H_{MM} \end{array}\right]$ with Protein-Protein, Protein-Membrane, Membrane-Protein and Membrane-Membrane terms. $H_{PP}$ is built as commonly done - pairing all atoms in the Protein within a given cutoff weigthed with spring constant $k_{PP}$, but we need to explicit the other parts (To be done in greater detail). 

In a nutshell, the coupling $H_{PM}$ involves the coupling between each membrane particle and any atoms from the protein within a given cutoff, weighted with a strong spring constant $k_{PM}$.

The membrane part $H_{MM}$ follows a slightly different definition as it constraints the *particle position*, not the *interparticle distance* as in the other terms. Something like $H_{MM}(i,j) = \frac{\partial E_{M}}{\partial x_{i}\partial x_{j}}$ where $E_{M} = \sum_{i} k_{i}|\mathbf{r}_{i}-\mathbf{r}_{i}^{(0)}|^{2}$.

Then the total Hessian is reduced to the effective protein Hessian $H_{eff} = H_{PP} - H_{PM}H_{MM}^{-1}H_{MP}$.

## Calculating the modes

Once the effective Hessian is built, things become normal again, and normal modes are obtained by solving the eigevanlue problem.

# Results

In [67]:
%matplotlib inline
import numpy as np                    # load numerical library
import numpy.ma as ma
from scipy.spatial import distance
from matplotlib import pyplot as plt  # load plotting library
from prody import *                   # load prody library
from pyvdwsurface import *            # load pyvdwsurface (installed from https://github.com/rmcgibbo/pyvdwsurface)

In [85]:
pdbs_folder  = 'pdbs'           # local folder where PDB files can be found
pdb_filename = '4FZ0.pdb'       # name of PDB considered for the analysis (must be found in pdbs_folder)
cutoff       = 15.0             # distance cutoff to couple pairs of atoms
gamma        = 1.0              # see ProDy
whole_selection         = 'protein'              # set of all atoms considered          
transmembrane_selection = whole_selection+' and (resnum 55:65 or resnum 430:440)'      # subset of whole_selection, for ligand
pdb = parsePDB(pdbs_folder+'/'+pdb_filename)

@> 10557 atoms and 1 coordinate set(s) were parsed in 0.08s.


In [91]:
skin = build_skin(pdb,selection=transmembrane_selection,write_xyz='skin.xyz')
#skin = build_skin(pdb,selection=transmembrane_selection)

Number of skin points:  1834


In [90]:
def build_skin(pdb,selection='protein',write_xyz=None):
    """ build_skin
    """
    crd = pdb.select(selection).getCoords()
    elements = [str.encode(x) for x in sel.getElements()]
    # first pass, we build all the skin
    skin = vdwsurface(crd, elements, scale_factor=5,density=0.1)
    # second pass, we remove parts that overlap with the rest of the protein
    crd_all = pdb.select('protein').getCoords()
    #skin = trimsurface(skin,crd_all)
    print("Number of skin points: ",skin.shape[0])
    if write_xyz is not None:
        save_xyz(write_xyz,skin)
    return skin

def trimsurface(skin,crd):
    """
    """
    # measure distance between all atoms in skin and all atoms in crd
    dist     = distance.cdist(skin,crd,'euclidean')
    # keep minimum distance to crd for all atoms in skin
    dist_min = np.amin(dist, axis=1)
    masked   = ma.masked_where(dist_min > 2., dist_min)
    mask     = np.repeat(masked,3,axis=0).reshape(skin.shape)
    print(mask.shape)
    new_skin = np.ma.masked_array(skin, mask)
    return np.ma.compressed(new_skin)
    
def save_xyz(filename,xyz):
    """ save_xyz
    """
    f = open(filename,'w+')
    f.write('{}'.format(xyz.shape[0])+"\r\n")
    f.write(" \r\n")
    for i in xyz:
        f.write("C "+'{}'.format(' '.join(map(str, i)))+"\r\n")
    f.close()


# Misc

In [ ]:
traj = pt.load(pdbs_folder+'/'+pdb_filename)
view = nv.show_pytraj(traj)
skin2shape(skin, view=view)
view

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
import nglview as nv
import pytraj as pt

def skin2shape(skin, view):
    shape = nv.shape.Shape(view=view)
    for i in skin:
        shape.add_sphere(i.tolist(), [1,0,0], 0.1)
 